# 13.1 运算符重载基础

Python 做了一些限制：
* 不能重载内置类型的运算符
* 不能新建运算符，只能重载现有的
* 某些运算符不能重载 —— is、and、or 和 not (不过位运算符 &、| 和 ~ 可以)

# 13.2 一元运算符

* \- (\_\_neg__) :一元取负算数运算符。如果 x 是 -2，那么 -x == 2
* \+ (\_\_pos__) :一元取正运算符，通常 x == +x，但也有一些例外。
* \~ (\_\_invert__)  :对整数按位取反，定义为 ~x == -(x+1)。如果 x 是 2，那么 ~x == -3

In [ ]:
# 实现 - + 的vector
import math

def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

def __pos__(self):
    return Vector(self)

def __neg__(self):
    return Vector(-x for x in self)

# 13.3 重载向量加法运算符

In [ ]:
# Vector.__add__ 方法 v1
import itertools

def __add__(self, other):
    # pairs 是一个生成器，生成(a, b)形式的元组，其中 a 来自 self，b 来自 other。如果它们长度不同，使用 fillvalue 填充较短的可迭代对象
    pairs = itertools.zip_longest(self, other, fillvalue = 0.0)
    return Vector(a + b for a,b in pairs)

v1 版的加法可以适配任何可迭代对象，但如果对调操作数(新内容为左值)，混合类型的加法就会失效

为了支持涉及不同类型的运算，Python 为中缀运算符特殊方法提供了特殊的分派机制。对表达式 a + b 来说，解释器会执行一下几步操作：
1. 如果 a 有 \_\_add__ 方法，而且返回值不是 NotImplemented，调用 a.\_\_add__(b)，返回结果
2. 如果 a 没有 \_\_add__ 方法，或 a.\_\_add__(b) 返回 NotImplemented，调用 b.\_\_radd__(a)，返回结果
3. 如果 b 没有 \_\_radd__ 方法，或 b.\_\_radd__(a) 返回 NotImplemented，抛出 TypeError，并在错误消息中指明操作数类型不支持

所以我们需要实现 Vector.\_\_radd__ 方法以支持对调操作数

In [ ]:
# Vector.__radd__ 方法支持
import itertools

def __add__(self, other):
    # pairs 是一个生成器，生成(a, b)形式的元组，其中 a 来自 self，b 来自 other。如果它们长度不同，使用 fillvalue 填充较短的可迭代对象
    pairs = itertools.zip_longest(self, other, fillvalue = 0.0)
    return Vector(a + b for a,b in pairs)

def __radd__(self, other):
    return self + other # radd 方法直接委托 add

这样实现后，Vector 可以处理任何具有数值元素的可迭代对象，但如果对象不可迭代就无法处理，或操作数是可迭代对象但他的元素不能与 Vector 中的浮点数元素相加

如果由于类型不兼容应该返回 NotImplemented，而不是抛出 TypeError。返回 NotImplemented 时，另一个操作数所属的类型还有机会执行运算，Python 会尝试调用反向方法。所以我们需要捕获异常并返回 NotImplemented

In [ ]:
# Vector.__add__ 修改版
import itertools

def __add__(self, other):
    # pairs 是一个生成器，生成(a, b)形式的元组，其中 a 来自 self，b 来自 other。如果它们长度不同，使用 fillvalue 填充较短的可迭代对象
    try:
        pairs = itertools.zip_longest(self, other, fillvalue = 0.0)
        return Vector(a + b for a,b in pairs)
    except TypeError:
        return NotImplemented

def __radd__(self, other):
    return self + other # radd 方法直接委托 add

# 13.4 重载标量乘法运算符 * 

我们需要实现 Vector([1, 2, 3]) * 10 = Vector([10.0, 20.0, 30.0])，并且适配反向方法

In [ ]:
def __mul__(self, scalar):
    return Vector(n * scalar for n in self)

def __rmul__(self, scalar):
    return self * scalar

这两个方法确实可用，但是提供不兼容的操作数时就会出问题。scalar 参数的值要是数字。不能使用复数，但可以是 int bool 甚至 fractions.Fraction 实例等标量

我们可以采用鸭子类型技术，在 __mul__ 方法捕获 TypeError。但我们可以使用白鹅类型

In [ ]:
import numbers
def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):
        return Vector(n * scalar for n in self)
    else:
        return NotImplemented

def __rmul__(self, scalar):
    return self * scalar

# 13.5 众多比较运算符

Python 解释器对众多比较运算符 (==、 !=、 >、 <、 >=、 <=) 的处理与前文类似，不过在两个方面有重大区别
* 正向和反向调用的是同一系列方法，只是把参数对调了，比如 大于等于 和 小于 是互相对应的
* 对 == 和 != 来说，如果反向调用失败。Python 会比较对象的 ID，而不抛出 TypeError

了解这些规则后，我们来改进 Vector.\_\_eq__ 方法的行为，它之前是这么定义的

In [ ]:
class Vector:
    # 省略

    def __eq__(self, other) -> bool:
        return (len(self) == len(self) and all(a == b for a,b in zip(self, other)))

In [ ]:
# 这样定义的缺陷，Vector 实例的分量与元组或其他任何可迭代对象的元组相等，那么对象也相等
va = Vector([1.0, 2.0, 3.0])
t3 = (1, 2, 3)
va == t3

因此，我们需要类型检查已确认第二个操作数为 Vector 实例 (或 Vector 子类的实例)

In [ ]:
class Vector:
    # 省略

    def __eq__(self, other) -> bool:
        if isinstance(other, Vector):
            return (len(self) == len(self) and all(a == b for a,b in zip(self, other)))
        else:
            return NotImplemented

# 13.6 增量赋值运算符

Vector 类已经支持增量赋值运算符 += 和 *= 了

如果一个类没有实现就地运算符，那增量运算符只是语法糖：a += b 等同于 a = a + b，会返回新的实例，但如果实现了就地运算符，那么 Python 解释器就会调用它，并且会修改左操作数而不是创建新的实例。

要注意的是，不可变类型一定不能实现就地特殊方法

与 + 相比，+= 运算符对第二个操作舒适更为宽容， + 运算符需要两个相同类型，而 += 运算符只需要右边操作数可以扩展到左边就行了

In [ ]:
# BingoCage 类
import random

class BingoCage(Tombola):   # 指明 BingoCage 类扩展 Tombola 类

    def __init__(self, items) -> None:
        self._randomizer = random.SystemRandom()    # random.SystemRandom 使用 os.urandom(...) 函数实现 random API。
        self._items = []
        self.load(items)    # 委托 load 方法实现初始加载
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)   # 没有使用 random.shuffle() 而是使用 SystemRandom 实例的 .shuffle() 方法
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [ ]:
# 实现就地运算的 AddableBingoCage
import itertools

class AddableBingoCage(BingoCage):

    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self